XLMRoberta, Mbert and Mdistilbert of multilingual 
Steps 1- 5 from preprocess to model fine tune.

Script 1: Preprocessing (for XLM-RoBERTa, mBERT, DistilBERT Multi)
Disk space issues so compressed the space in below script 1 and 2

In [1]:
import pandas as pd
from transformers import (
    XLMRobertaTokenizer,
    BertTokenizer,
    DistilBertTokenizer
)
from datasets import Dataset
from sklearn.model_selection import train_test_split
import gc

# --- Preprocessing ---

# Load Jigsaw dataset
data_path = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv"
df = pd.read_csv(data_path)
df = df.drop_duplicates(subset=["comment_text"])
df = df[["comment_text", "toxic"]].dropna()

# Balance classes
toxic_df = df[df["toxic"] == 1]
non_toxic_df = df[df["toxic"] == 0].sample(n=len(toxic_df), random_state=42)
balanced_df = pd.concat([toxic_df, non_toxic_df]).sample(frac=1, random_state=42)

# Split: 70% train, 15% val, 15% test
train_df, temp_df = train_test_split(balanced_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Rename 'toxic' to 'labels' for Trainer compatibility
train_df = train_df.rename(columns={"toxic": "labels"})
val_df = val_df.rename(columns={"toxic": "labels"})
test_df = test_df.rename(columns={"toxic": "labels"})

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenization function with optimization
def tokenize_dataset(dataset, tokenizer, max_length=32):  # Reduced from 64 to 32
    def tokenize_function(examples):
        return tokenizer(examples["comment_text"], padding="max_length", truncation=True, max_length=max_length)
    # Remove original text after tokenization to save space
    tokenized = dataset.map(tokenize_function, batched=True, remove_columns=["comment_text"])
    return tokenized

# Initialize tokenizers for multilingual models
xlm_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
bert_multi_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
distilbert_multi_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-multilingual-cased")

# Tokenize datasets for each multilingual model
train_xlm = tokenize_dataset(train_dataset, xlm_tokenizer)
val_xlm = tokenize_dataset(val_dataset, xlm_tokenizer)
test_xlm = tokenize_dataset(test_dataset, xlm_tokenizer)

train_bert_multi = tokenize_dataset(train_dataset, bert_multi_tokenizer)
val_bert_multi = tokenize_dataset(val_dataset, bert_multi_tokenizer)
test_bert_multi = tokenize_dataset(test_dataset, bert_multi_tokenizer)

train_distilbert_multi = tokenize_dataset(train_dataset, distilbert_multi_tokenizer)
val_distilbert_multi = tokenize_dataset(val_dataset, distilbert_multi_tokenizer)
test_distilbert_multi = tokenize_dataset(test_dataset, distilbert_multi_tokenizer)

# Save tokenized datasets with compression
def save_compressed(dataset, path):
    dataset.save_to_disk(path, num_proc=1)  # Single process to avoid memory spikes
    gc.collect()  # Clear memory after saving

save_compressed(train_xlm, "/kaggle/working/preprocessed/train_xlm")
save_compressed(val_xlm, "/kaggle/working/preprocessed/val_xlm")
save_compressed(test_xlm, "/kaggle/working/preprocessed/test_xlm")

save_compressed(train_bert_multi, "/kaggle/working/preprocessed/train_bert_multi")
save_compressed(val_bert_multi, "/kaggle/working/preprocessed/val_bert_multi")
save_compressed(test_bert_multi, "/kaggle/working/preprocessed/test_bert_multi")

save_compressed(train_distilbert_multi, "/kaggle/working/preprocessed/train_distilbert_multi")
save_compressed(val_distilbert_multi, "/kaggle/working/preprocessed/val_distilbert_multi")
save_compressed(test_distilbert_multi, "/kaggle/working/preprocessed/test_distilbert_multi")

print("Preprocessing completed. Tokenized datasets saved to /kaggle/working/preprocessed/")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/4589 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/21411 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4588 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4589 [00:00<?, ? examples/s]

Preprocessing completed. Tokenized datasets saved to /kaggle/working/preprocessed/


Script 2: Fine-Tuning (for XLM-RoBERTa, mBERT, DistilBERT Multi)

In [2]:
import torch
from transformers import (
    XLMRobertaForSequenceClassification,
    BertForSequenceClassification,
    DistilBertForSequenceClassification,
    XLMRobertaTokenizer,
    BertTokenizer,
    DistilBertTokenizer,
    Trainer, TrainingArguments
)
from datasets import Dataset
import gc

# Load tokenized datasets
train_xlm = Dataset.load_from_disk("/kaggle/working/preprocessed/train_xlm")
val_xlm = Dataset.load_from_disk("/kaggle/working/preprocessed/val_xlm")
train_bert_multi = Dataset.load_from_disk("/kaggle/working/preprocessed/train_bert_multi")
val_bert_multi = Dataset.load_from_disk("/kaggle/working/preprocessed/val_bert_multi")
train_distilbert_multi = Dataset.load_from_disk("/kaggle/working/preprocessed/train_distilbert_multi")
val_distilbert_multi = Dataset.load_from_disk("/kaggle/working/preprocessed/val_distilbert_multi")

# Shared training arguments with minimal disk usage
base_training_args = TrainingArguments(
    output_dir="/kaggle/working/models",  # Single shared directory
    eval_strategy="epoch",
    save_strategy="no",  # Disable checkpointing to save space
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=False,  # Avoid extra model loading
    report_to="none"
)

# Function to fine-tune and save a model
def fine_tune_model(model, tokenizer, train_dataset, val_dataset, model_name):
    trainer = Trainer(
        model=model,
        args=base_training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )
    trainer.train()
    # Save only the final model and tokenizer to the shared directory
    model.save_pretrained(f"/kaggle/working/models/{model_name}")
    tokenizer.save_pretrained(f"/kaggle/working/models/{model_name}")
    # Clear memory
    del trainer, model
    gc.collect()
    torch.cuda.empty_cache()

# Fine-tune Multilingual BERT (mBERT)
bert_multi_model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=2)
bert_multi_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
fine_tune_model(bert_multi_model, bert_multi_tokenizer, train_bert_multi, val_bert_multi, "bert_multi_finetuned")
del bert_multi_tokenizer  # Tokenizer can be reused later if needed
gc.collect()

# Fine-tune Multilingual DistilBERT
distilbert_multi_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=2)
distilbert_multi_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-multilingual-cased")
fine_tune_model(distilbert_multi_model, distilbert_multi_tokenizer, train_distilbert_multi, val_distilbert_multi, "distilbert_multi_finetuned")
del distilbert_multi_tokenizer
gc.collect()

# Fine-tune XLM-RoBERTa (Multilingual) - Uncomment if needed
#xlm_model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)
#xlm_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
#fine_tune_model(xlm_model, xlm_tokenizer, train_xlm, val_xlm, "xlm_finetuned")
#del xlm_tokenizer
#gc.collect()

print("Fine-tuning completed. Models saved to /kaggle/working/models/")

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fine-tuning completed. Models saved to /kaggle/working/models/


In [3]:
import torch
from transformers import (
    XLMRobertaForSequenceClassification,
    BertForSequenceClassification,
    DistilBertForSequenceClassification,
    XLMRobertaTokenizer,
    BertTokenizer,
    DistilBertTokenizer,
    Trainer, TrainingArguments
)
from datasets import Dataset
import gc

# Load tokenized datasets
train_xlm = Dataset.load_from_disk("/kaggle/working/preprocessed/train_xlm")
val_xlm = Dataset.load_from_disk("/kaggle/working/preprocessed/val_xlm")

# Shared training arguments with minimal disk usage
base_training_args = TrainingArguments(
    output_dir="/kaggle/working/models",  # Single shared directory
    eval_strategy="epoch",
    save_strategy="no",  # Disable checkpointing to save space
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=False,  # Avoid extra model loading
    report_to="none"
)


# Function to fine-tune and save a model
def fine_tune_model(model, tokenizer, train_dataset, val_dataset, model_name):
    trainer = Trainer(
        model=model,
        args=base_training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )
    trainer.train()
    # Save only the final model and tokenizer to the shared directory
    model.save_pretrained(f"/kaggle/working/models/{model_name}")
    tokenizer.save_pretrained(f"/kaggle/working/models/{model_name}")
    # Clear memory
    del trainer, model
    gc.collect()
    torch.cuda.empty_cache()

# Fine-tune XLM-RoBERTa (Multilingual) - Uncomment if needed
xlm_model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)
xlm_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
fine_tune_model(xlm_model, xlm_tokenizer, train_xlm, val_xlm, "xlm_finetuned")
del xlm_tokenizer
gc.collect()



model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0

Script 3: Evaluation (for XLM-RoBERTa, mBERT, DistilBERT Multi)

In [4]:
import pandas as pd
from transformers import (
    XLMRobertaForSequenceClassification,
    BertForSequenceClassification,
    DistilBertForSequenceClassification,
    Trainer, TrainingArguments
)
from datasets import Dataset
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    roc_auc_score, confusion_matrix
)
import numpy as np

# --- Evaluation ---

# Compute metrics function (updated to include AUC and confusion matrix)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # Predicted classes
    probs = pred.predictions[:, 1]  # Probabilities for positive class (toxic)

    # Standard metrics
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    
    # AUC-ROC
    auc = roc_auc_score(labels, probs)
    
    # Confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "auc": auc,
        "true_positives": tp,
        "true_negatives": tn,
        "false_positives": fp,
        "false_negatives": fn
    }

# Load test datasets
test_xlm = Dataset.load_from_disk("/kaggle/working/preprocessed/test_xlm")
test_bert_multi = Dataset.load_from_disk("/kaggle/working/preprocessed/test_bert_multi")
test_distilbert_multi = Dataset.load_from_disk("/kaggle/working/preprocessed/test_distilbert_multi")

# Load fine-tuned models
xlm_model = XLMRobertaForSequenceClassification.from_pretrained("/kaggle/working/models/xlm_finetuned")
bert_multi_model = BertForSequenceClassification.from_pretrained("/kaggle/working/models/bert_multi_finetuned")
distilbert_multi_model = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/models/distilbert_multi_finetuned")

# Evaluation training args
eval_args = TrainingArguments(
    output_dir="/kaggle/working/models",
    eval_strategy="epoch",
    per_device_eval_batch_size=8,
    report_to="none"
)

# Evaluate XLM-RoBERTa (Multilingual)
xlm_trainer = Trainer(
    model=xlm_model,
    args=eval_args,
    eval_dataset=test_xlm,
    compute_metrics=compute_metrics
)
xlm_eval = xlm_trainer.evaluate()

# Evaluate Multilingual BERT (mBERT)
bert_multi_trainer = Trainer(
    model=bert_multi_model,
    args=eval_args,
    eval_dataset=test_bert_multi,
    compute_metrics=compute_metrics
)
bert_multi_eval = bert_multi_trainer.evaluate()

# Evaluate Multilingual DistilBERT
distilbert_multi_trainer = Trainer(
    model=distilbert_multi_model,
    args=eval_args,
    eval_dataset=test_distilbert_multi,
    compute_metrics=compute_metrics
)
distilbert_multi_eval = distilbert_multi_trainer.evaluate()

# Compile and save results
results = {
    "Model": ["XLM-RoBERTa", "mBERT", "DistilBERT (Multi)"],
    "Accuracy": [xlm_eval["eval_accuracy"], bert_multi_eval["eval_accuracy"], distilbert_multi_eval["eval_accuracy"]],
    "F1": [xlm_eval["eval_f1"], bert_multi_eval["eval_f1"], distilbert_multi_eval["eval_f1"]],
    "Precision": [xlm_eval["eval_precision"], bert_multi_eval["eval_precision"], distilbert_multi_eval["eval_precision"]],
    "Recall": [xlm_eval["eval_recall"], bert_multi_eval["eval_recall"], distilbert_multi_eval["eval_recall"]],
    "AUC": [xlm_eval["eval_auc"], bert_multi_eval["eval_auc"], distilbert_multi_eval["eval_auc"]],
    "True Positives": [xlm_eval["eval_true_positives"], bert_multi_eval["eval_true_positives"], distilbert_multi_eval["eval_true_positives"]],
    "True Negatives": [xlm_eval["eval_true_negatives"], bert_multi_eval["eval_true_negatives"], distilbert_multi_eval["eval_true_negatives"]],
    "False Positives": [xlm_eval["eval_false_positives"], bert_multi_eval["eval_false_positives"], distilbert_multi_eval["eval_false_positives"]],
    "False Negatives": [xlm_eval["eval_false_negatives"], bert_multi_eval["eval_false_negatives"], distilbert_multi_eval["eval_false_negatives"]]
}

results_df = pd.DataFrame(results)
results_df.to_csv("/kaggle/working/multilingual_model_performance_with_auc_cm.csv", index=False)
print(results_df)

print("Evaluation completed. Results with AUC and confusion matrix saved to /kaggle/working/multilingual_model_performance_with_auc_cm.csv")

                Model  Accuracy        F1  Precision    Recall       AUC  \
0         XLM-RoBERTa  0.901722  0.902275   0.902471  0.902080  0.963818   
1               mBERT  0.904990  0.904260   0.916741  0.892114  0.964314   
2  DistilBERT (Multi)  0.893223  0.893893   0.893506  0.894281  0.955340   

   True Positives  True Negatives  False Positives  False Negatives  
0            2082            2056              225              226  
1            2059            2094              187              249  
2            2064            2035              246              244  
Evaluation completed. Results with AUC and confusion matrix saved to /kaggle/working/multilingual_model_performance_with_auc_cm.csv


Testing on 3 youtube comments datasets

In [5]:
import pandas as pd
import torch
from transformers import (
    XLMRobertaForSequenceClassification, XLMRobertaTokenizer,
    BertForSequenceClassification, BertTokenizer,
    DistilBertForSequenceClassification, DistilBertTokenizer
)
from datasets import Dataset
import os

# Disk space check function (optional, for Kaggle monitoring)
def check_disk_space():
    stat = os.statvfs("/kaggle/working")
    free_space = stat.f_frsize * stat.f_bavail / (1024 ** 3)  # GB
    print(f"Free disk space: {free_space:.2f} GB")
    return free_space

# Step 1: Load Excel file and convert to CSV
excel_file_path = "/kaggle/input/random-multi-youtube-comments-scrapped/Youtube comments scrapper_Testing File.xlsx"
comments_df = pd.read_excel(excel_file_path)

# Check for 'comment' column and adjust if necessary
if "comment" not in comments_df.columns:
    print("Column 'comment' not found. Available columns:", comments_df.columns)
    # Assuming the first column might be comments if misnamed
    comments_df = comments_df.rename(columns={comments_df.columns[0]: "comment"})
else:
    print(f"Loaded Excel file from {excel_file_path} with {len(comments_df)} comments.")

# Step 2: Convert to Hugging Face Dataset
comments_dataset = Dataset.from_pandas(comments_df[["comment"]])

# Step 3: Tokenizers for the three fine-tuned multilingual models
xlm_tokenizer = XLMRobertaTokenizer.from_pretrained("/kaggle/working/models/xlm_finetuned")
bert_multi_tokenizer = BertTokenizer.from_pretrained("/kaggle/working/models/bert_multi_finetuned")
distilbert_multi_tokenizer = DistilBertTokenizer.from_pretrained("/kaggle/working/models/distilbert_multi_finetuned")

# Step 4: Tokenization function
def tokenize_dataset(dataset, tokenizer, max_length=32):  # Using 32 as per your multilingual preprocessing
    def tokenize_function(examples):
        return tokenizer(examples["comment"], padding="max_length", truncation=True, max_length=max_length)
    return dataset.map(tokenize_function, batched=True)

# Tokenize datasets for all three models
comments_xlm = tokenize_dataset(comments_dataset, xlm_tokenizer)
comments_bert_multi = tokenize_dataset(comments_dataset, bert_multi_tokenizer)
comments_distilbert_multi = tokenize_dataset(comments_dataset, distilbert_multi_tokenizer)

# Step 5: Load the fine-tuned multilingual models
xlm_model = XLMRobertaForSequenceClassification.from_pretrained("/kaggle/working/models/xlm_finetuned")
bert_multi_model = BertForSequenceClassification.from_pretrained("/kaggle/working/models/bert_multi_finetuned")
distilbert_multi_model = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/models/distilbert_multi_finetuned")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to device
xlm_model.to(device)
bert_multi_model.to(device)
distilbert_multi_model.to(device)

# Step 6: Predict toxicity function
def predict_toxicity(model, tokenizer, dataset, device):
    model.eval()
    predictions = []
    probabilities = []
    comments = dataset["comment"]
    for comment in comments:
        inputs = tokenizer(comment, return_tensors="pt", padding=True, truncation=True, max_length=32)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        pred = probs.argmax(-1).item()
        toxic_prob = probs[0][1].item()
        predictions.append(pred)
        probabilities.append(toxic_prob)
    return predictions, probabilities

# Step 7: Get predictions for each model
print("Predicting with XLM-RoBERTa...")
xlm_preds, xlm_probs = predict_toxicity(xlm_model, xlm_tokenizer, comments_xlm, device)

print("Predicting with mBERT...")
bert_multi_preds, bert_multi_probs = predict_toxicity(bert_multi_model, bert_multi_tokenizer, comments_bert_multi, device)

print("Predicting with DistilBERT Multi...")
distilbert_multi_preds, distilbert_multi_probs = predict_toxicity(distilbert_multi_model, distilbert_multi_tokenizer, comments_distilbert_multi, device)

# Step 8: Combine and save results
results_df = pd.DataFrame({
    "Comment": comments_df["comment"],
    "XLMRoBERTa_Pred": xlm_preds,
    "XLMRoBERTa_Toxic_Prob": xlm_probs,
    "mBERT_Pred": bert_multi_preds,
    "mBERT_Toxic_Prob": bert_multi_probs,
    "DistilBERTMulti_Pred": distilbert_multi_preds,
    "DistilBERTMulti_Toxic_Prob": distilbert_multi_probs
})

results_csv_path = "/kaggle/working/combined_multilingual_model_predictions_RMYC.csv"
results_df.to_csv(results_csv_path, index=False)
print(f"Predictions saved to {results_csv_path}")
print(results_df.head(10))  # Show first 10 rows for inspection

# Optional: Check disk space
check_disk_space()

Loaded Excel file from /kaggle/input/random-multi-youtube-comments-scrapped/Youtube comments scrapper_Testing File.xlsx with 563 comments.


Map:   0%|          | 0/563 [00:00<?, ? examples/s]

Map:   0%|          | 0/563 [00:00<?, ? examples/s]

Map:   0%|          | 0/563 [00:00<?, ? examples/s]

Predicting with XLM-RoBERTa...
Predicting with mBERT...
Predicting with DistilBERT Multi...
Predictions saved to /kaggle/working/combined_multilingual_model_predictions_RMYC.csv
                                             Comment  XLMRoBERTa_Pred  \
0         Learning to speak is always useful. Great!                0   
1  Vocal Exercises begin at 7:50 :   1. Raise arm...                0   
2  Imagine someone being late and walking in at 8:49                1   
3  4:01 "Tempered with love, honesty is a great t...                0   
4  "If you wish people with love its really hard ...                0   
5  i feel like youtube recommendations are person...                0   
6                      I like his head it is shining                0   
7  Very fantastic video.❤❤  I came across this vi...                0   
8  0:13 Intro 0:33 7 deadly sins of speaking 2:45...                0   
9  The trick is, whenever you're talking to peopl...                1   

   XLMRoBERTa_Toxi

17.3154296875

In [6]:
import pandas as pd
import torch
from transformers import (
    XLMRobertaForSequenceClassification, XLMRobertaTokenizer,
    BertForSequenceClassification, BertTokenizer,
    DistilBertForSequenceClassification, DistilBertTokenizer
)
from datasets import Dataset
import os

# Disk space check function (optional, for Kaggle monitoring)
def check_disk_space():
    stat = os.statvfs("/kaggle/working")
    free_space = stat.f_frsize * stat.f_bavail / (1024 ** 3)  # GB
    print(f"Free disk space: {free_space:.2f} GB")
    return free_space

# Step 1: Load CSV file from Kaggle environment
csv_file_path = "/kaggle/input/youtube-toxicity-data/youtoxic_english_1000.csv"  # Replace with your actual CSV file path
comments_df = pd.read_csv(csv_file_path)

# Check for 'comment' column and adjust if necessary
if "Text" not in comments_df.columns:
    print("Column 'comment' not found. Available columns:", comments_df.columns)
    # Assuming the first column might be comments if misnamed
    comments_df = comments_df.rename(columns={comments_df.columns[0]: "comment"})
else:
    print(f"Loaded Excel file from {excel_file_path} with {len(comments_df)} comments.")

# Step 2: Convert to Hugging Face Dataset
comments_dataset = Dataset.from_pandas(comments_df[["Text"]])

# Step 3: Tokenizers for the three fine-tuned multilingual models
xlm_tokenizer = XLMRobertaTokenizer.from_pretrained("/kaggle/working/models/xlm_finetuned")
bert_multi_tokenizer = BertTokenizer.from_pretrained("/kaggle/working/models/bert_multi_finetuned")
distilbert_multi_tokenizer = DistilBertTokenizer.from_pretrained("/kaggle/working/models/distilbert_multi_finetuned")

# Step 4: Tokenization function
def tokenize_dataset(dataset, tokenizer, max_length=32):  # Using 32 as per your multilingual preprocessing
    def tokenize_function(examples):
        return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=max_length)
    return dataset.map(tokenize_function, batched=True)

# Tokenize datasets for all three models
comments_xlm = tokenize_dataset(comments_dataset, xlm_tokenizer)
comments_bert_multi = tokenize_dataset(comments_dataset, bert_multi_tokenizer)
comments_distilbert_multi = tokenize_dataset(comments_dataset, distilbert_multi_tokenizer)

# Step 5: Load the fine-tuned multilingual models
xlm_model = XLMRobertaForSequenceClassification.from_pretrained("/kaggle/working/models/xlm_finetuned")
bert_multi_model = BertForSequenceClassification.from_pretrained("/kaggle/working/models/bert_multi_finetuned")
distilbert_multi_model = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/models/distilbert_multi_finetuned")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to device
xlm_model.to(device)
bert_multi_model.to(device)
distilbert_multi_model.to(device)

# Step 6: Predict toxicity function
def predict_toxicity(model, tokenizer, dataset, device):
    model.eval()
    predictions = []
    probabilities = []
    comments = dataset["Text"]
    for comment in comments:
        inputs = tokenizer(comment, return_tensors="pt", padding=True, truncation=True, max_length=32)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        pred = probs.argmax(-1).item()
        toxic_prob = probs[0][1].item()
        predictions.append(pred)
        probabilities.append(toxic_prob)
    return predictions, probabilities

# Step 7: Get predictions for each model
print("Predicting with XLM-RoBERTa...")
xlm_preds, xlm_probs = predict_toxicity(xlm_model, xlm_tokenizer, comments_xlm, device)

print("Predicting with mBERT...")
bert_multi_preds, bert_multi_probs = predict_toxicity(bert_multi_model, bert_multi_tokenizer, comments_bert_multi, device)

print("Predicting with DistilBERT Multi...")
distilbert_multi_preds, distilbert_multi_probs = predict_toxicity(distilbert_multi_model, distilbert_multi_tokenizer, comments_distilbert_multi, device)

# Step 8: Combine and save results
results_df = pd.DataFrame({
    "Comment": comments_df["Text"],
    "XLMRoBERTa_Pred": xlm_preds,
    "XLMRoBERTa_Toxic_Prob": xlm_probs,
    "mBERT_Pred": bert_multi_preds,
    "mBERT_Toxic_Prob": bert_multi_probs,
    "DistilBERTMulti_Pred": distilbert_multi_preds,
    "DistilBERTMulti_Toxic_Prob": distilbert_multi_probs
})

results_csv_path = "/kaggle/working/combined_multilingual_model_predictions_YTD.csv"
results_df.to_csv(results_csv_path, index=False)
print(f"Predictions saved to {results_csv_path}")
print(results_df.head(10))  # Show first 10 rows for inspection

# Optional: Check disk space
check_disk_space()

Loaded Excel file from /kaggle/input/random-multi-youtube-comments-scrapped/Youtube comments scrapper_Testing File.xlsx with 1000 comments.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Predicting with XLM-RoBERTa...
Predicting with mBERT...
Predicting with DistilBERT Multi...
Predictions saved to /kaggle/working/combined_multilingual_model_predictions_YTD.csv
                                             Comment  XLMRoBERTa_Pred  \
0  If only people would just take a step back and...                0   
1  Law enforcement is not trained to shoot to app...                1   
2  \nDont you reckon them 'black lives matter' ba...                1   
3  There are a very large number of people who do...                0   
4  The Arab dude is absolutely right, he should h...                1   
5  here people his facebook is https://www.facebo...                1   
6  Check out this you tube post. "Black man goes ...                1   
7  I would LOVE to see this pussy go to Staten Is...                1   
8                        I agree with the protestor.                0   
9   mike browns father was made to say that boooshit                1   

   XLMRoBERTa_Toxic

17.31519317626953

In [7]:
import pandas as pd
import torch
from transformers import (
    XLMRobertaForSequenceClassification, XLMRobertaTokenizer,
    BertForSequenceClassification, BertTokenizer,
    DistilBertForSequenceClassification, DistilBertTokenizer
)
from datasets import Dataset
import os

# Disk space check function (optional, for Kaggle monitoring)
def check_disk_space():
    stat = os.statvfs("/kaggle/working")
    free_space = stat.f_frsize * stat.f_bavail / (1024 ** 3)  # GB
    print(f"Free disk space: {free_space:.2f} GB")
    return free_space

# Step 1: Load CSV file from Kaggle environment
csv_file_path = "/kaggle/input/most-liked-comments-on-youtube/youtube_dataset.csv"  # Replace with your actual CSV file path
comments_df = pd.read_csv(csv_file_path)

# Check for 'comment' column and adjust if necessary
if "Comment" not in comments_df.columns:
    print("Column 'comment' not found. Available columns:", comments_df.columns)
    # Assuming the first column might be comments if misnamed
    comments_df = comments_df.rename(columns={comments_df.columns[0]: "comment"})
else:
    print(f"Loaded Excel file from {excel_file_path} with {len(comments_df)} comments.")

# Step 2: Convert to Hugging Face Dataset
comments_dataset = Dataset.from_pandas(comments_df[["Comment"]])

# Step 3: Tokenizers for the three fine-tuned multilingual models
xlm_tokenizer = XLMRobertaTokenizer.from_pretrained("/kaggle/working/models/xlm_finetuned")
bert_multi_tokenizer = BertTokenizer.from_pretrained("/kaggle/working/models/bert_multi_finetuned")
distilbert_multi_tokenizer = DistilBertTokenizer.from_pretrained("/kaggle/working/models/distilbert_multi_finetuned")

# Step 4: Tokenization function
def tokenize_dataset(dataset, tokenizer, max_length=32):  # Using 32 as per your multilingual preprocessing
    def tokenize_function(examples):
        return tokenizer(examples["Comment"], padding="max_length", truncation=True, max_length=max_length)
    return dataset.map(tokenize_function, batched=True)

# Tokenize datasets for all three models
comments_xlm = tokenize_dataset(comments_dataset, xlm_tokenizer)
comments_bert_multi = tokenize_dataset(comments_dataset, bert_multi_tokenizer)
comments_distilbert_multi = tokenize_dataset(comments_dataset, distilbert_multi_tokenizer)

# Step 5: Load the fine-tuned multilingual models
xlm_model = XLMRobertaForSequenceClassification.from_pretrained("/kaggle/working/models/xlm_finetuned")
bert_multi_model = BertForSequenceClassification.from_pretrained("/kaggle/working/models/bert_multi_finetuned")
distilbert_multi_model = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/models/distilbert_multi_finetuned")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to device
xlm_model.to(device)
bert_multi_model.to(device)
distilbert_multi_model.to(device)

# Step 6: Predict toxicity function
def predict_toxicity(model, tokenizer, dataset, device):
    model.eval()
    predictions = []
    probabilities = []
    comments = dataset["Comment"]
    for comment in comments:
        inputs = tokenizer(comment, return_tensors="pt", padding=True, truncation=True, max_length=32)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        pred = probs.argmax(-1).item()
        toxic_prob = probs[0][1].item()
        predictions.append(pred)
        probabilities.append(toxic_prob)
    return predictions, probabilities

# Step 7: Get predictions for each model
print("Predicting with XLM-RoBERTa...")
xlm_preds, xlm_probs = predict_toxicity(xlm_model, xlm_tokenizer, comments_xlm, device)

print("Predicting with mBERT...")
bert_multi_preds, bert_multi_probs = predict_toxicity(bert_multi_model, bert_multi_tokenizer, comments_bert_multi, device)

print("Predicting with DistilBERT Multi...")
distilbert_multi_preds, distilbert_multi_probs = predict_toxicity(distilbert_multi_model, distilbert_multi_tokenizer, comments_distilbert_multi, device)

# Step 8: Combine and save results
results_df = pd.DataFrame({
    "Comment": comments_df["Comment"],
    "XLMRoBERTa_Pred": xlm_preds,
    "XLMRoBERTa_Toxic_Prob": xlm_probs,
    "mBERT_Pred": bert_multi_preds,
    "mBERT_Toxic_Prob": bert_multi_probs,
    "DistilBERTMulti_Pred": distilbert_multi_preds,
    "DistilBERTMulti_Toxic_Prob": distilbert_multi_probs
})

results_csv_path = "/kaggle/working/combined_multilingual_model_predictions_YD.csv"
results_df.to_csv(results_csv_path, index=False)
print(f"Predictions saved to {results_csv_path}")
print(results_df.head(10))  # Show first 10 rows for inspection

# Optional: Check disk space
check_disk_space()

Loaded Excel file from /kaggle/input/random-multi-youtube-comments-scrapped/Youtube comments scrapper_Testing File.xlsx with 19300 comments.


Map:   0%|          | 0/19300 [00:00<?, ? examples/s]

Map:   0%|          | 0/19300 [00:00<?, ? examples/s]

Map:   0%|          | 0/19300 [00:00<?, ? examples/s]

Predicting with XLM-RoBERTa...
Predicting with mBERT...
Predicting with DistilBERT Multi...
Predictions saved to /kaggle/working/combined_multilingual_model_predictions_YD.csv
                                             Comment  XLMRoBERTa_Pred  \
0  The people who liked this comment is officiall...                0   
1           - Wait, it's 7B views\n- Always has been                0   
2  *Teacher: What is the population of the Earth?...                0   
3  Let's be honest this wasn't your recommendatio...                0   
4  Types Of People:\n10% Enjoying Song \n90% Chec...                0   
5  3.2 Million comments if you find mine your a l...                1   
6  claim your “here before 7 billion” tickets her...                0   
7  The ones who are NOT from Tik-Tok can like thi...                0   
8   Song: spanish\nComments: English\nHotel: trivago                0   
9                                    Kimler burda😂🥰🌹                1   

   XLMRoBERTa_Toxic_

17.311519622802734